In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train_tmp, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

In [6]:
if len(instances_train_tmp) > config.size:
    instances_train = np.random.choice(instances_train_tmp, config.size, replace=False)
else:
    instances_train = instances_train_tmp

In [7]:
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [8]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [9]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [10]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [11]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    time_start = time.time()
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

    if epoch % 10 == 0:
        time_log = float(time.time() - time_start)
        
        # validate
        loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
        ppl_train = np.exp(np.mean(ppls_train))
        loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

        # test
        if ppl_dev < ppl_min:
            ppl_min = ppl_dev
            loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
            saver.save(sess, config.path_model, global_step=global_step_log)
            cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
            update_checkpoint(config, checkpoint, global_step_log)

        # visualize topic
        topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
        topics_freq_idxs = bow_idxs[topics_freq_indices]
        topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
        topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
        descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
        recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}

        depth_specs = get_topic_specialization(sess, model, instances_test)
        hierarchical_affinities = get_hierarchical_affinity(sess, model)

        # log
        clear_output()
        log_series = pd.Series([time_log, epoch, ct, \
                '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                '%.2f'%loss_dev, ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                '%.2f'%loss_test, ppl_test, \
                '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                index=log_df.columns)
        log_df.loc[global_step_log] = log_series
        display(log_df)
        cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
        print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

        # update tree
        if not config.static:
            config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
            if update_tree_flg:
                print(config.tree_idxs)
                name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                if 'sess' in globals(): sess.close()
                model = HierarchicalNeuralTopicModel(config)
                sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                saver = tf.train.Saver(max_to_keep=1)

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                           VALID:              \
           Time   Ep  Ct    LOSS  PPL     NLL    KL   REG    LOSS         PPL   
15     2.550660    0  14  120.05  954  119.05  0.80  0.20  110.57  718.108459   
165    0.274164   10  14  112.10  602  111.62  0.44  0.04  106.18  555.139832   
315    0.154749   20  14  111.37  578  111.01  0.34  0.02  106.04  551.448853   
465    0.144234   30  14  111.02  567  110.68  0.32  0.02  106.03  551.380493   
615    0.151257   40  14  110.74  558  110.35  0.37  0.02  106.16  555.617310   
765    0.144923   50  14  110.46  551  110.00  0.45  0.01  106.22  558.770264   
915    0.374013   60  14  110.21  544  109.66  0.54  0.01  106.19  558.289795   
1065   0.157421   70  14  109.98  537  109.34  0.62  0.01  106.18  559.189575   
1215   0.165452   80  14  109.76  532  109.05  0.70  0.01  106.18  559.275696   
1365   0.151236   90  14  109.56  526  108.78  0.78  0.01  106.23  561.753845   
1515   0.166877  100  14  109.38  522  108.52  0.85  0.01  106.27  563.373291   
1665   0.144790  110  14  109.21  517  108.28  0.92  0.01  106.34  566.256897   
1815   0.224149  120  14  109.05  513  108.06  0.98  0.01  106.42  568.951843   
1965   0.152056  130  14  108.89  509  107.85  1.04  0.01  106.53  572.373657   
2115   0.259275  140  14  108.75  506  107.65  1.09  0.01  106.67  576.614502   
2265   0.157436  150  14  108.62  502  107.46  1.15  0.01  106.80  580.470032   
2415   0.166195  160  14  108.49  499  107.29  1.20  0.01  106.94  584.773132   
2565   0.351473  170  14  108.37  496  107.12  1.24  0.01  107.06  588.278870   
2715   0.155682  180  14  108.26  494  106.97  1.28  0.01  107.18  592.096069   
2865   0.303168  190  14  108.15  491  106.82  1.32  0.01  107.29  595.011536   
3015   0.148409  200  14  108.05  489  106.68  1.36  0.01  107.40  598.370056   
3165   0.149569  210  14  107.96  486  106.56  1.39  0.01  107.51  601.766479   
3315   0.148753  220  14  107.86  484  106.43  1.43  0.01  107.62  605.230347   
3465   0.152273  230  14  107.78  482  106.32  1.46  0.01  107.73  608.630310   
3615   0.171214  240  14  107.70  480  106.20  1.49  0.01  107.82  611.901367   
3765   0.155348  250  14  107.62  479  106.10  1.51  0.01  107.92  614.917297   
3915   0.481062  260  14  107.54  477  106.00  1.54  0.01  108.01  618.055664   
4065   0.162003  270  14  107.47  475  105.90  1.56  0.01  108.09  620.689209   
4215   0.318703  280  14  107.40  473  105.81  1.59  0.01  108.17  623.598877   
4365   0.270910  290  14  107.33  472  105.72  1.61  0.01  108.25  626.017456   
...         ...  ...  ..     ...  ...     ...   ...   ...     ...         ...   
10515  0.169545  700  14  105.87  439  103.85  2.01  0.00  110.55  708.842224   
10665  0.180032  710  14  105.85  438  103.83  2.01  0.00  110.59  710.451233   
10815  0.173665  720  14  105.83  438  103.80  2.02  0.00  110.63  712.027588   
10965  0.175965  730  14  105.81  437  103.78  2.02  0.00  110.67  713.574768   
11115  0.169037  740  14  105.79  437  103.76  2.03  0.00  110.71  715.085083   
11265  0.171515  750  14  105.77  436  103.73  2.03  0.00  110.75  716.562378   
11415  0.229824  760  14  105.75  436  103.71  2.03  0.00  110.78  717.998169   
11565  0.378109  770  14  105.73  436  103.69  2.04  0.00  110.82  719.388855   
11715  0.163876  780  14  105.71  435  103.67  2.04  0.00  110.85  720.752319   
11865  0.164937  790  14  105.70  435  103.65  2.04  0.00  110.89  722.081543   
12015  0.167006  800  14  105.68  434  103.63  2.05  0.00  110.92  723.421448   
12165  0.165921  810  14  105.66  434  103.61  2.05  0.00  110.95  724.736267   
12315  0.177822  820  14  105.65  434  103.59  2.05  0.00  110.99  726.058960   
12465  0.161151  830  14  105.63  433  103.57  2.06  0.00  111.02  727.318909   
12615  0.315250  840  14  105.61  433  103.55  2.06  0.00  111.05  728.605652   
12765  0.170099  850  14  105.60  433  103.53  2.06  0.00  111.08  729.828247   
12915  0.172877  860  14  105.58  432  103.51  2.07  0.00

0 R: 1.000 P: 0.341 carry ! pockets back quality strap lot nice work bought
   1 R: 0.275 P: 0.120 pocket room nice power pockets mouse carry inside larger cord
     11 R: 0.093 P: 0.093 ; & sleeve pro protection zipper inside quality nice bit
     13 R: 0.052 P: 0.052 ipad sleeve chromebook ! protection quality perfectly protect nice air
     14 R: 0.011 P: 0.011 ! perfect ... money mouse put love durable needed time
   2 R: 0.112 P: 0.034 usb nice works cord purchase fine power problem padding pleased
     22 R: 0.078 P: 0.078 ! mac air protect protection nice scratches color bought price
   5 R: 0.065 P: 0.027 ! bit months 'm corners feels fabric year bought recommend
     51 R: 0.021 P: 0.021 ; & color ! love inside lot black water nice
     52 R: 0.017 P: 0.017 ! love color perfect work school durable recommend red college
   3 R: 0.207 P: 0.096 bottom top cover nice apple ! cut logo feet broke
     31 R: 0.111 P: 0.111 ! cover color love pro perfectly keyboard hard easily nice
{0

TRAIN:                           VALID:              \
           Time   Ep  Ct    LOSS  PPL     NLL    KL   REG    LOSS         PPL   
15     2.550660    0  14  120.05  954  119.05  0.80  0.20  110.57  718.108459   
165    0.274164   10  14  112.10  602  111.62  0.44  0.04  106.18  555.139832   
315    0.154749   20  14  111.37  578  111.01  0.34  0.02  106.04  551.448853   
465    0.144234   30  14  111.02  567  110.68  0.32  0.02  106.03  551.380493   
615    0.151257   40  14  110.74  558  110.35  0.37  0.02  106.16  555.617310   
765    0.144923   50  14  110.46  551  110.00  0.45  0.01  106.22  558.770264   
915    0.374013   60  14  110.21  544  109.66  0.54  0.01  106.19  558.289795   
1065   0.157421   70  14  109.98  537  109.34  0.62  0.01  106.18  559.189575   
1215   0.165452   80  14  109.76  532  109.05  0.70  0.01  106.18  559.275696   
1365   0.151236   90  14  109.56  526  108.78  0.78  0.01  106.23  561.753845   
1515   0.166877  100  14  109.38  522  108.52  0.85  0.01  106.27  563.373291   
1665   0.144790  110  14  109.21  517  108.28  0.92  0.01  106.34  566.256897   
1815   0.224149  120  14  109.05  513  108.06  0.98  0.01  106.42  568.951843   
1965   0.152056  130  14  108.89  509  107.85  1.04  0.01  106.53  572.373657   
2115   0.259275  140  14  108.75  506  107.65  1.09  0.01  106.67  576.614502   
2265   0.157436  150  14  108.62  502  107.46  1.15  0.01  106.80  580.470032   
2415   0.166195  160  14  108.49  499  107.29  1.20  0.01  106.94  584.773132   
2565   0.351473  170  14  108.37  496  107.12  1.24  0.01  107.06  588.278870   
2715   0.155682  180  14  108.26  494  106.97  1.28  0.01  107.18  592.096069   
2865   0.303168  190  14  108.15  491  106.82  1.32  0.01  107.29  595.011536   
3015   0.148409  200  14  108.05  489  106.68  1.36  0.01  107.40  598.370056   
3165   0.149569  210  14  107.96  486  106.56  1.39  0.01  107.51  601.766479   
3315   0.148753  220  14  107.86  484  106.43  1.43  0.01  107.62  605.230347   
3465   0.152273  230  14  107.78  482  106.32  1.46  0.01  107.73  608.630310   
3615   0.171214  240  14  107.70  480  106.20  1.49  0.01  107.82  611.901367   
3765   0.155348  250  14  107.62  479  106.10  1.51  0.01  107.92  614.917297   
3915   0.481062  260  14  107.54  477  106.00  1.54  0.01  108.01  618.055664   
4065   0.162003  270  14  107.47  475  105.90  1.56  0.01  108.09  620.689209   
4215   0.318703  280  14  107.40  473  105.81  1.59  0.01  108.17  623.598877   
4365   0.270910  290  14  107.33  472  105.72  1.61  0.01  108.25  626.017456   
...         ...  ...  ..     ...  ...     ...   ...   ...     ...         ...   
10515  0.169545  700  14  105.87  439  103.85  2.01  0.00  110.55  708.842224   
10665  0.180032  710  14  105.85  438  103.83  2.01  0.00  110.59  710.451233   
10815  0.173665  720  14  105.83  438  103.80  2.02  0.00  110.63  712.027588   
10965  0.175965  730  14  105.81  437  103.78  2.02  0.00  110.67  713.574768   
11115  0.169037  740  14  105.79  437  103.76  2.03  0.00  110.71  715.085083   
11265  0.171515  750  14  105.77  436  103.73  2.03  0.00  110.75  716.562378   
11415  0.229824  760  14  105.75  436  103.71  2.03  0.00  110.78  717.998169   
11565  0.378109  770  14  105.73  436  103.69  2.04  0.00  110.82  719.388855   
11715  0.163876  780  14  105.71  435  103.67  2.04  0.00  110.85  720.752319   
11865  0.164937  790  14  105.70  435  103.65  2.04  0.00  110.89  722.081543   
12015  0.167006  800  14  105.68  434  103.63  2.05  0.00  110.92  723.421448   
12165  0.165921  810  14  105.66  434  103.61  2.05  0.00  110.95  724.736267   
12315  0.177822  820  14  105.65  434  103.59  2.05  0.00  110.99  726.058960   
12465  0.161151  830  14  105.63  433  103.57  2.06  0.00  111.02  727.318909   
12615  0.315250  840  14  105.61  433  103.55  2.06  0.00  111.05  728.605652   
12765  0.170099  850  14  105.60  433  103.53  2.06  0.00  111.08  729.828247   
12915  0.172877  860  14  105.58  432  103.51  2.07  0.00

0 R: 1.000 P: 0.384 carry ! pockets back lot quality strap nice work love
   1 R: 0.248 P: 0.115 pocket room pockets mouse power nice carry inside strap cord
     11 R: 0.082 P: 0.082 ; & sleeve pro protection zipper inside nice quality bit
     13 R: 0.042 P: 0.042 ipad sleeve chromebook ! protection perfectly quality protect inside nice
     12 R: 0.009 P: 0.009 ! perfect ... money mouse put love durable needed time
   2 R: 0.096 P: 0.033 usb nice works cord fine purchase power problem job padding
     22 R: 0.063 P: 0.063 mac ! air protect protection nice scratches color bought perfectly
   3 R: 0.060 P: 0.018 ! bit months 'm corners feels year fabric bought recommend
     31 R: 0.024 P: 0.024 & ; ! color love inside lot black nice colors
     32 R: 0.018 P: 0.018 ! love color perfect work school durable recommend red college
   4 R: 0.212 P: 0.094 bottom top cover nice apple ! cut logo feet broke
     41 R: 0.118 P: 0.118 ! cover color love pro perfectly hard keyboard easily nice
